In [ ]:
!pip install langchain --quiet
!pip install -qU langchain-deepseek

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.4 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ['DEEPSEEK_API_KEY'] = userdata.get("DEEPSEEK_API_KEY")

#Basic Prompt

In [ ]:
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(model='deepseek-chat')
result=llm.invoke("What are the ways to improve the mood? Give me the output in bullet points")
print(result)

content="Here are some effective ways to improve your mood:\n\n- **Exercise Regularly**: Engage in physical activities like walking, jogging, yoga, or dancing to release endorphins and reduce stress.  \n- **Practice Gratitude**: Write down or reflect on things you’re grateful for to shift your focus to positive aspects of life.  \n- **Connect with Others**: Spend time with friends, family, or loved ones to foster a sense of belonging and support.  \n- **Get Enough Sleep**: Prioritize restful sleep to recharge your body and mind.  \n- **Listen to Music**: Play your favorite uplifting songs to boost your mood.  \n- **Spend Time in Nature**: Go for a walk in a park, hike, or simply sit outside to enjoy fresh air and sunlight.  \n- **Meditate or Practice Mindfulness**: Focus on the present moment to reduce anxiety and improve emotional well-being.  \n- **Engage in a Hobby**: Do something you enjoy, like painting, cooking, gardening, or reading.  \n- **Eat Mood-Boosting Foods**: Consume foo

# RAG Application

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

In [ ]:
!pip install langchain-community --quiet
!pip install tiktoken --quiet
!pip install chromadb --quiet
!pip install pypdf --quiet
!pip install langchain-openai --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.llms import Cohere

In [ ]:
# Step-1:Document Loading
from langchain.document_loaders import PyPDFLoader
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Banking_System_Doc/BASEL.pdf
loader = PyPDFLoader("BASEL.pdf")
pages = loader.load()
print(len(pages))

#Step-2:Split the data into Chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

#Step-3: Creating Embeddings
from langchain.embeddings import OpenAIEmbeddings, CohereEmbeddings
embeddings = OpenAIEmbeddings()

#Step-4: Storing in a Vector DB
#!rm -rf basel_norms_db
basel_norms_db= Chroma.from_documents(chunks,
                                    embeddings,
                                    persist_directory="basel_norms_db"
                          )
basel_norms_db.persist()

--2025-02-11 11:26:03--  https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Banking_System_Doc/BASEL.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1260197 (1.2M) [application/octet-stream]
Saving to: ‘BASEL.pdf.2’

BASEL.pdf.2         100%[===================>]   1.20M  --.-KB/s    in 0.01s   

2025-02-11 11:26:04 (105 MB/s) - ‘BASEL.pdf.2’ saved [1260197/1260197]

77
521


<ipython-input-29-724cd4ef971e>:25: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  basel_norms_db.persist()


In [ ]:
#Step-5 Retrieval

retriever = basel_norms_db.as_retriever()
result=retriever.get_relevant_documents("What percentage is the minimum Capital Requirements",
                                        )
print(result)
print(len(result))
print([i.metadata for i in result])

[Document(metadata={'agenda item1': '', 'author': 'Basel Committee on Banking Supervision', 'comments': '', 'company': '', 'contenttype': 'Document', 'creationdate': '2011-05-31T11:52:06+02:00', 'creator': 'Acrobat PDFMaker 9.1 for Word', 'description0': '', 'doc #': '', 'keywords': '', 'meeting': '2010 - 138th Meeting of the BCBS November-December 2010', 'moddate': '2019-11-14T17:33:29+01:00', 'page': 67, 'page_label': '68', 'producer': 'Acrobat Distiller 9.4.0 (Windows)', 'sort order': '0.110000000000000', 'source': 'BASEL.pdf', 'subject': 'Full text of "Basel III: A global regulatory framework for more resilient banks and banking systems - post BCBS meeting - revised version", June 2011', 'title': 'Basel III: A global regulatory framework for more resilient banks and banking systems - post BCBS meeting - revised version June 2011', 'total_pages': 77}, page_content='capital) \nMinimum Capital Conservation Ratios \n(expressed as a percentage of earnings) \n4.5% -  5.75% 100% \n>5.75% 

In [ ]:
#Step-6 Retrieval Q and A
from langchain.chains import RetrievalQA
#llm=OpenAI()
#llm=Cohere()
llm = ChatDeepSeek(model='deepseek-chat')


Q_AChain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",# It takes a list of documents, inserts them all into a prompt
    retriever=retriever
)

query="What percentage is the minimum Capital Requirements?"
docs=Q_AChain({"query":query})
print(docs["result"])

 The minimum Capital Requirements are expressed as a percentage of earnings, with the lowest being 4.5% and the highest being 9.5%. The specific percentage required for each bank will depend on their earnings and other factors.
